In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [67]:
def u(x):
    return np.log(1 + x * x) + np.sin(x)

In [90]:
class Solver:

    def __init__(self, h, CFL, u):
        self.L = 1
        self.T = 1
        self.h = h
        self.CFL = CFL
        self.u = u

        self.tau = self.CFL * self.h

        self.NX = int(self.L / self.h + 1)
        self.TX = int(self.T / self.tau + 1) 

        self.x_i = np.array([(i - 1) * self.h for i in range(1, self.NX + 1)])
        self.t_i = np.array([(i - 1) * self.tau for i in range(1, self.TX + 1)])

        self.u_n_m = self.u(self.x_i)
        self.u_next_m = self.u_n_m.copy()

        self.analytic_x_i = np.linspace(0, 1, 11)
        self.analytic_u = np.sin(self.analytic_x_i) + np.log(1 + np.power((self.analytic_x_i - 1), 2))
        self.analytic_all = np.sin(self.x_i) + np.log(1 + np.power((self.x_i - 1), 2))

        self.u_result = []
        self.delta_u = []

    def solve(self):
        for t in self.t_i:
            self.u_next_m[0] = np.log(1 + t * t)
            self.u_next_m[1] = self.u_n_m[0] + self.h * (1 - (2*t)/(1+t*t)) + (np.power(self.h, 2) / 2) * ((2*(1 - t*t))/np.power((t*t + 1), 2)) + (np.power(self.h, 3) / 6) * (- 1 - (4*t*(np.power(t,2) - 3))/(np.power((1 + t*t), 3)))
            self.u_next_m[2] = self.u_n_m[0] + 2 * self.h * (1 - (2*t)/(1+t*t)) + 4 * (np.power(self.h, 2) / 2) * ((2*(1 - t*t))/np.power((t*t + 1), 2)) + 8 * (np.power(self.h, 3) / 6) * (- 1 - (4*t*(np.power(t,2) - 3))/(np.power((1 + t*t), 3)))
            for i in range(3, self.NX):
                self.u_next_m[i] = self.u_n_m[i] \
                        + (self.CFL / 6) * (2*self.u_n_m[i-3] - 9*self.u_n_m[i-2] + 18*self.u_n_m[i-1] - 11*self.u_n_m[i]) \
                        + (np.power(self.CFL, 2) / 2) * (-self.u_n_m[i-3] + 4*self.u_n_m[i-2] - 5*self.u_n_m[i-1] + 2*self.u_n_m[i]) \
                        - (np.power(self.CFL, 3) / 6) * (-self.u_n_m[i-3] + 3*self.u_n_m[i-2] - 3*self.u_n_m[i-1] + self.u_n_m[i]) \
                        + self.tau*np.cos(self.x_i[i]) + 0.5*np.power(self.tau, 2)*np.sin(self.x_i[i]) - (np.power(self.tau, 3)/6)*np.cos(self.x_i[i])
            self.u_n_m = self.u_next_m.copy()
    def display(self):
        for ind, x in enumerate(self.analytic_x_i):
            i, = np.where(np.isclose(self.x_i, x))
            self.u_result.append(float(self.u_n_m[i]))
            self.delta_u.append(np.abs(self.u_result[ind] - self.analytic_u[ind]))
        self.df = pd.DataFrame({"x": self.analytic_x_i, "u аналит.": self.analytic_u, "u числ. ": self.u_result ,"delta u": self.delta_u})
        pd.options.display.float_format = '{:.6E}'.format
        display(self.df)
        max_err = self.df['delta u'].max()
        print(f'{max_err:.2E}')

    def displayMaxErr(self):
        print(f'cfl = {self.CFL}\tmax error = {np.max(np.abs(self.analytic_all - self.u_n_m))}')

In [81]:
for cfl in [0.9, 1., 1.1, 1.9, 2., 2.1, 2.9, 3., 3.1]:
    s = Solver(0.001, cfl, u)
    s.solve()
    s.displayMaxErr()

cfl = 0.9	max error = 4.3080443207332565e+22
cfl = 1.0	max error = 0.0009999998352102635
cfl = 1.1	max error = 0.001217644900655901
cfl = 1.9	max error = 0.002553256045043417
cfl = 2.0	max error = 0.001999998666666891
cfl = 2.1	max error = 1.3059271094794496e+16
cfl = 2.9	max error = 0.005631300807231621
cfl = 3.0	max error = 0.0039999882996558656
cfl = 3.1	max error = 2.735106598077746e+69


In [82]:
s_r = Solver(0.001, 1, u)
s_r.solve()
s_r.display()

/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_79727/493017134.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.u_result.append(float(self.u_n_m[i]))


,x,u аналит.,u числ.,delta u
0,0.000000E+00,6.931472E-01,6.931472E-01,0.000000E+00
1,1.000000E-01,6.931603E-01,6.921656E-01,9.946476E-04
2,2.000000E-01,6.933656E-01,6.923896E-01,9.760093E-04
3,3.000000E-01,6.942963E-01,6.933560E-01,9.402840E-04
4,4.000000E-01,6.969030E-01,6.960197E-01,8.833881E-04
5,5.000000E-01,7.025691E-01,7.017677E-01,8.014367E-04
6,6.000000E-01,7.130625E-01,7.123710E-01,6.915245E-04
7,7.000000E-01,7.303954E-01,7.298426E-01,5.527533E-04
8,8.000000E-01,7.565768E-01,7.561895E-01,3.872756E-04
9,9.000000E-01,7.932772E-01,7.930763E-01,2.009299E-04


9.95E-04


In [91]:
for h in [0.01, 0.01/2, 0.01/4]:
    s = Solver(h, 1, u)
    s.solve()
    print(h)
    s.displayMaxErr()

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35000000000000003
0.36
0.37
0.38
0.39
0.4
0.41000000000000003
0.42
0.43
0.44
0.45
0.46
0.47000000000000003
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.5700000000000001
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.6900000000000001
0.7000000000000001
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.8200000000000001
0.8300000000000001
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.9400000000000001
0.9500000000000001
0.96
0.97
0.98
0.99
1.0
0.01
cfl = 1	max error = 0.009999852264976683
0.0
0.005
0.01
0.015
0.02
0.025
0.03
0.035
0.04
0.045
0.05
0.055
0.06
0.065
0.07
0.075
0.08
0.085
0.09
0.095
0.1
0.105
0.11
0.115
0.12
0.125
0.13
0.135
0.14
0.145
0.15
0.155
0.16
0.165
0.17
0.17500000000000002
0.18
0.185
0.19
0.195
0.2
0.20500000000000002
0.21
0.215
0.22
0.225
0.23
0.23500000000000001
0.24
0.245
0.